In [1]:
import os
# If cwd is not the same as the script's directory, change to the script's directory
if os.getcwd() != os.path.dirname('/home/joe/HPC/particleBox'):
    os.chdir('/home/joe/HPC/particleBox')
    print("Set working directory to script's directory: " + os.getcwd())


import matplotlib.pyplot as plt
import numpy as np

Set working directory to script's directory: /home/joe/HPC/particleBox


In [ ]:
# Import traj files
# Formatted as:
# Line 1: Number of particles
# Line 2: time
# Line 3 to N+2: data for each particle (position, velocity, etc.) for each time step. First entry is a string for particle type, followed by 12 numerical values (e.g. x, y, z, vx, vy, vz, ax, ay, az, fx, fy, fz)
# Line N+3: Repeat for next time step
def read_traj_file(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()
        num_particles = int(lines[0].strip())
        time = (lines[1].strip())
        data = np.zeros((1000, num_particles, 13)) # Assuming 13 columns of data per particle (e.g. str(ID), x, y, z, vx, vy, vz, ax, ay, az, fx, fy, fz)
        for t in range(1000):
            for p in range(num_particles):
                line_index = 2 + t * (num_particles + 2) + p
                line = lines[line_index].strip().split()
                # Remove the first character (e.g. 'a') from the particle type string and convert to int
                particle_ID = int(line[0][1:]) # Assuming particle type is in the format 'a0', 'a1', etc.
                data[t, p, 0] = int(particle_ID) # Store the particle ID as an integer in the first column
                data[t, p, 1:] = list(map(float, line[1:])) # Convert the rest of the data to floats
        
    return data


traj_data = read_traj_file('fast.xyz')
print(traj_data.shape) # Should be (num_timesteps, num_particles, 13)
print(traj_data[0, 1, :]) # Data for first particle at first time step

(1000, 1000, 13)


array([ 1.0000000e+00, -2.9373486e+01, -2.4987155e+01,  1.0000000e+00,
        1.9680000e-02, -2.2716000e-02,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
        0.0000000e+00])

In [20]:
newtraj_data = read_traj_file('testfast.xyz')
oldtraj_data = read_traj_file('testslow.xyz')

In [21]:
"""
Sanity Check for New Method:
1. Check that the dataline for a few particles are identical between the old and new methods for the same time step.    
"""


# Check that the dataline for a few particles are identical between the old and new methods for the same time step.

for t in range(10): # Check the first 10 time steps
    for ID in range(5): # Check the first 5 IDs
        
        # Find the index of the particle with the given ID in the new and old data
        new_index = np.where(newtraj_data[t, :, 0] == ID)[0]
        old_index = np.where(oldtraj_data[t, :, 0] == ID)[0]
        
        if len(new_index) == 0 or len(old_index) == 0:
            print(f"Particle with ID {ID} not found in time step {t}")
            continue
        
        # Compare the data for the particle with the given ID in the new and old data
        if np.array_equal(newtraj_data[t, new_index, :], oldtraj_data[t, old_index, :]):
            print(f"Particle with ID {ID} has identical data in time step {t}")
        else:
            print(f"Particle with ID {ID} has different data in time step {t}\n")
            print(f"New data: {newtraj_data[t, new_index, :]}")
            print(f"Old data: {oldtraj_data[t, old_index, :]}")
            
            


Particle with ID 0 has identical data in time step 0
Particle with ID 1 has identical data in time step 0
Particle with ID 2 has identical data in time step 0
Particle with ID 3 has identical data in time step 0
Particle with ID 4 has identical data in time step 0
Particle with ID 0 has different data in time step 1

New data: [[ 0.0000000e+00 -4.6655414e+01 -1.7001867e+01  1.0000000e+00
  -2.7788000e-02  3.2890000e-03  0.0000000e+00 -3.8006440e+00
   1.3490180e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00]]
Old data: [[ 0.0000000e+00 -4.6658672e+01 -1.7000072e+01  1.0000000e+00
  -7.3573000e-02  1.4740000e-02  0.0000000e+00 -4.7699490e+00
   1.5499340e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00]]
Particle with ID 1 has different data in time step 1

New data: [[ 1.0000000e+00 -2.9369342e+01 -2.4982173e+01  1.0000000e+00
   3.7480000e-03  4.1491000e-02  0.0000000e+00 -2.9941000e+00
  -2.6118470e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
 